In [106]:
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

In [107]:
# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):

    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})    
    response.encoding ='utf8'   
    
    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}

    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())
    
    
    
    for script in page.find_all("script"):
        if 'vendor":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')
            
        if '"price_info":{"price":' in str(script):
            a = str(script)
            data_dict['price'] = re.search(r'"price_info":{"price":\d*', a)[0][22:]
            
        if '{"mileage":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]
        
        if 'engine_type":' in str(script):
            a = str(script)
            data_dict['fuelType'] = re.search(r'engine_type":".*?"', a)[0][14:].strip('"')
        
        if '"mark":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['brand'] = re.search(r'"mark":".*?"', a)[0][8:].strip('"')
            
        if 'transmission":"' in str(script):
            a = str(script)
            data_dict['vehicleTransmission'] = re.search(r'transmission":".*?"', a)[0][15:].strip('"')
            
        if 'body_type":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vehicleConfiguration'] = re.search(r'body_type":".*?"', a)[0][12:].strip('"')
            
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]
        
        if 'equipment":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]
            
        if '"model_info":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')
        
        if '"power"' in str(script):
            a = str(script)
            data_dict['enginePower'] = re.search(r'"power".*?"', a)[0][8:11]
            
        if 'doors_count":' in str(script):
            a = str(script)
            data_dict['numberOfDoors'] = re.search(r'doors_count":.*?"', a)[0][13:14]
        
            
            
    for li in page.find('li', class_= 'CardInfoRow CardInfoRow_color'):
        data_dict['color'] = li.text
    for div in page.find('div', class_='CardDescription__textInner'):
        data_dict['description'] = div.text
    for lis in page.find('li', class_= 'CardInfoRow CardInfoRow_engine'):
        data_dict['engineDisplacement'] = lis.text[:3]
    for lii in page.find('li', class_= 'CardInfoRow CardInfoRow_year'):
        data_dict['productionDate'] = lii.text
    for liii in page.find('li', class_= 'CardInfoRow CardInfoRow_wheel'):
        data_dict['Руль'] = liii.text
    for la in page.find('li', class_= 'CardInfoRow CardInfoRow_ownersCount'):
        data_dict['Владельцы'] = la.text[:1]
    for lb in page.find('li', class_= 'CardInfoRow CardInfoRow_state'):
        data_dict['Состояние'] = lb.text
    for lc in page.find('li', class_= 'CardInfoRow CardInfoRow_pts'):
        data_dict['ПТС'] = lc.text
    for ld in page.find('li', class_="CardInfoRow CardInfoRow_drive"):
        data_dict['Привод'] = ld.text
    for le in page.find('li', class_="CardInfoRow CardInfoRow_customs"):
        data_dict['Таможня'] = le.text
        
        
            
            

            
    return data_dict



def extraction_links(url):
    links_list =[] 
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})  
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list

In [108]:
%%time

df = pd.DataFrame()

url_link_list = [] # список страниц по годам и номерам от 1 до 99

ranges = [range(1970, 2021), range(1, 100)]

# index[0] - year, index[1] - page 
for index in itertools.product(*ranges):
    # формируем страницы со списками объявлений
    url_links = (f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    url_link_list.append(url_links)
    
links_list = [] # список списков ссылок на объявления c одной страницы таблицы объявлений

#for url_links in url_link_list:
# извлекаем в links_list список ссылок на объявления
try:
        #links_list = extraction_links(url_links)
    links_list = Parallel(n_jobs = 2)(delayed(extraction_links)(url_links) for url_links in url_link_list)
except:
    pass 

ads_dict_list = [] # список словарей содержимого объявлений

for links in links_list:
    try:
        ads_dict_list = Parallel(n_jobs = 2)(delayed(parsing_page_one_ad)(ad_url) for ad_url in links)
    except:
        pass
    
    for ad in ads_dict_list:
        try:
            df = df.append(ad, ignore_index=True)
        except:
            pass


Wall time: 7h 9min 8s


In [109]:
df.to_csv('abc.csv', encoding = 'utf-8', index=False)

In [110]:
data = pd.read_csv('abc.csv')

C:\Users\ddypr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [115]:
data.

Index(['brand', 'car_url', 'color', 'complectation_dict', 'description',
       'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType',
       'mileage', 'model_info', 'model_name', 'numberOfDoors',
       'parsing_unixtime', 'price', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня'],
      dtype='object')

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58078 entries, 0 to 58077
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 58078 non-null  object 
 1   car_url               58078 non-null  object 
 2   color                 58078 non-null  object 
 3   complectation_dict    58049 non-null  object 
 4   description           58076 non-null  object 
 5   engineDisplacement    58078 non-null  object 
 6   enginePower           58078 non-null  object 
 7   equipment_dict        58078 non-null  object 
 8   fuelType              58078 non-null  object 
 9   mileage               52787 non-null  float64
 10  model_info            58078 non-null  object 
 11  model_name            58078 non-null  object 
 12  numberOfDoors         58078 non-null  int64  
 13  parsing_unixtime      58078 non-null  float64
 14  price                 58015 non-null  float64
 15  productionDate     